# DataSource

Node definition for defining access and structure of data sources. The base class for data sources is `podpac.data.DataSource`.

The child class must implement the method `get_data`.
This method defines how to select data from the source given input PODPAC Coordinates.

The child class often implements the method `get_native_coordinates`.
This method defines defines how to select the native coordinates from the source and returns PODPAC Coordinates.

## Array Example

In [1]:
from podpac.data import DataSource
from podpac import Coordinates
import traitlets as tl     # traitlets is used to enforce types
import numpy as np

class MyDataType(DataSource):
    
    source = tl.Instance(np.ndarray)   # input "source" to this data type must be a numpy ndarray
                                       # DataSource includes input "native_coordinates" and enforces Coordinates type
    coordinates = tl.Instance(Coordinates).tag(attr=True)  # Coordinates describing this source
    
    def get_data(self, coordinates, coordinates_index):
        # this will run when the node is evaluated
        
        # use the index of the coordinates to select data from the "source"
        idx = coordinates_index
        data = self.source[idx]
        
        # Create a UnitsDataArray from coordinates and data from "source"
        uda = self.create_output_array(coordinates, data=data)
        
        return uda

In [2]:
# make a node from your DataSource
data = np.random.rand(5, 5)   # random 5x5 grid of data
coordinates = Coordinates([np.linspace(0, 5, 5), np.linspace(0, 5, 5)], dims=['lat', 'lon'])

node = MyDataType(source=data, coordinates=coordinates)
node

<MyDataType()>

In [3]:
# evaluate the node at coordinates in the top left of the grid
coords = Coordinates([[0, 1, 2], [0, 1, 2]], dims=['lat', 'lon'])
output = node.eval(coords)
output

INFO:numexpr.utils:NumExpr defaulting to 8 threads.


<xarray.UnitsDataArray (lat: 3, lon: 3)>
array([[0.16121145, 0.46074652, 0.57088041],
       [0.9019238 , 0.65131438, 0.71072657],
       [0.85214093, 0.70138685, 0.94792403]])
Coordinates:
  * lat      (lat) float64 0.0 1.0 2.0
  * lon      (lon) float64 0.0 1.0 2.0
Attributes:
    layer_style:   <podpac.core.style.Style object at 0x000001FCC26CE388>
    crs:           EPSG:4326
    geotransform:  (-0.5, 1.0, 0.0, -0.5, 0.0, 1.0)

## DataFrame Example

In [4]:
from podpac.data import DataSource
from podpac import Coordinates

import traitlets as tl     # traitlets is used to enforce types
import numpy as np
import pandas as pd

class OtherType(DataSource):
    
    source = tl.Instance(pd.DataFrame)   # input "source" to this data type must be a pandas DataFrame
    lat_key = tl.Unicode()               # where to find "lat" data in this dataframe
    lon_key = tl.Unicode()               # where to find "lon" in this dataframe
    data_key = tl.Unicode()              # where to find specific data of interest in this dataframe
    
    def get_coordinates(self):
        # this will return native coordinates as a podpac.Coordinates class
        
        lat = self.source[self.lat_key]
        lon = self.source[self.lon_key]
        
        # since this is tabular dat, our coordinates are point data, or "stacked"
        # in podpac we use the "_" to refer to stacked coordinates
        return Coordinates([(lat, lon)], dims=['lat_lon'])
    
    
    def get_data(self, coordinates, coordinates_index):
        # this will run when the node is evaluated
        
        # use the index of the coordinates to select data from the "source"
        idx = coordinates_index
        data = self.source[self.data_key].iloc[idx]
        
        # Create a UnitsDataArray from coordinates and data from "source"
        uda = self.create_output_array(coordinates, data=data)
        
        return uda

In [5]:
# make a node from your DataSource
data = pd.DataFrame(data={
    'values': np.random.rand(5),
    'lon_val': np.linspace(0, 5, 5),
    'lat_val': np.linspace(0, 5, 5)
})

node = OtherType(source=data,
                 lat_key='lat_val',
                 lon_key='lon_val',
                 data_key='values'
                )
node

<OtherType()>

In [6]:
# evaluate the node at coordinates in the top left of the grid
coords = Coordinates([([0, 1, 2], [0, 1, 2])], dims=['lat_lon'])
output = node.eval(coords)
output

<xarray.UnitsDataArray (lat_lon: 3)>
array([0.24954415, 0.26733937, 0.53771143])
Coordinates:
  * lat_lon  (lat_lon) MultiIndex
  - lat      (lat_lon) float64 0.0 1.0 2.0
  - lon      (lat_lon) float64 0.0 1.0 2.0
Attributes:
    layer_style:  <podpac.core.style.Style object at 0x000001FCC27834C8>
    crs:          EPSG:4326